In [1]:
from channel import *
import tda 
client = tda.auth.client_from_token_file('kiran.json', config.TD_CLIENT_ID)
import matplotlib.dates
import sys,os
import numpy as np

Purpose:

    Check a set of candles for a double bottom and return whether it is present.

Arguments:

    candles(pandas DataFrame) - DataFrame of the candles requested.

    revlen(int) - Number of candles required to classify a trend as a reversal

    variance(float) - The difference between the lows allowed as a percent of the first bottom.

Returns:

    result(bool) - True if the candles show a double bottomm pattern and false otherwise.

    argDict(dict) - A dictionary of arguments containing the take profit and stop loss.

    condCandles(pandas DataFrame) - The candles after the pattern has completed to check for the take profit or stop loss.    

In [ ]:
def double_bottom_identifier(candles:pd.DataFrame,revlen:int,variance:float):
    firstBottom = -1
    secondBottom = -1
    top = -1
    pos = 0
    numCandles = candles.shape[0]
    argDict = {}
    #Iterate through the candles until the chart reverses upwards the first time and holds for at least revlen candles
    while pos<numCandles:
        if candles.iloc[pos].low<min(candles.iloc[0:min(-1,pos-numCandles+1+revlen)].low):
            firstBottom = candles.iloc[pos]
            break
        pos+=1
    #Iterate through the remaining candles until the chart reverses downwards and holds for at least revlen candles
    while pos<numCandles:
        if candles.iloc[pos].high>max(candles.iloc[0:min(-1,pos-numCandles+1+revlen)].high):
            top = candles.iloc[pos]
            break
        pos+=1
    #Iterate through the remaining candles until the chart reverses a second time and holds for at least revlen candles
    while pos<numCandles:
        if candles.iloc[pos].low<min(candles.iloc[0:min(-1,pos-numCandles+1+revlen)].low):
            secondBottom = candles.iloc[pos]
            break
        pos+=1
    condCandles = candles.iloc[pos:]
    if abs(firstBottom.low-secondBottom.low)/firstBottom<variance:
        argDict["lowLine"] = mean([firstBottom.low,secondBottom.low])
        argDict["neckLine"] = top.high
        result = True
    else:
        result = False
    return result,argDict,condCandles

Purpose:

    Given a DataFrame of candles and a dictionary with the neckLine and lowLine identify the take profit and stop loss and return 0 if the pattern hits the stop loss, 1 if it hits neither the take profit or stop loss, and 2 if the pattern hits the take profit.

Arguments:

    candles(DataFrame) - The candles to check for the take profit or stop loss.

    argDict(dict) - A dictionary containing the neckLine and lowLine for the pattern.

Returns:

    ret(int) - 0,1, or 2 to indicate what the pattern hit.

    argDict(dict) - Dictionary with the neckLine and lowLine and the buy and sell locations for the showGraphs function.

In [ ]:
def double_bottom_conditional(candles:pd.DataFrame, argDict:dict):
    neckLine = argDict["neckLine"]
    lowLine = argDict["lowLine"]
    buy = -1
    sell = -1
    for candle in candles.iterrows():
        if candle.close>neckLine:
            buy = candle.datetime
        if 